#### 1. Contexto de Negócio (Business Understanding)
Objetivo: Criar uma visão unificada e calculada do pedido para determinar o Lucro Bruto Unitário (Margem de Contribuição) por transação. Esta métrica é vital para um Marketplace Two-Sided (conectando lojas, clientes e motoristas) porque permite à gestão otimizar preços, comissões, e repasses (P&L), garantindo a sustentabilidade da plataforma.

Desafio: O dado do Marketplace é naturalmente disperso em três fontes principais (Pedidos, Pagamentos e Logística), que precisam ser unidas e padronizadas.

#### 2. Camada BRONZE: Ingestão e Padronização (Data Preparation - Raw)
A Camada Bronze serve como a Zona de Aterrissagem (Landing Zone) e é a primeira etapa de preparação, garantindo a fidelidade dos dados brutos e a auditoria.

 Justificativas Técnicas
Imutabilidade: Os dados são carregados exatamente como vieram dos CSVs, preservando o histórico da fonte.

Correção de Chaves: O foco principal aqui foi resolver o [UNRESOLVED_COLUMN] e o desalinhamento das chaves do marketplace:

Mapeamento de delivery_order_id e payment_order_id para o campo canônico order_id.

Correção de case-sensitivity (ex: DELIVERED vs delivered) para evitar perdas de dados nas camadas superiores.

#### Etapas & Benefícios

| Ação | Etapa Técnica | Benefício Estratégico |
| :--- | :--- | :--- |
| Ingestão | Uso de COPY INTO e FILEFORMAT = CSV. | Auditabilidade: Cria tabelas rastreáveis (Bronze), permitindo reverter ou auditar qualquer transformação futura contra a fonte original. |
| Padronização | Uso de SELECT * AS novo_nome nos comandos COPY INTO. | Coerência: Garante que todas as tabelas de fatos usem uma chave de junção comum (order_id) antes da modelagem complexa. |
| Data Type | Tipagem inicial como STRING. | Resiliência: Tolerância a dados sujos (NULLs, formatos ruins) sem quebrar o pipeline, deixando a limpeza para a Camada Silver. |

#### 3. Camada SILVER: Unificação e Cálculo da UE (Data Preparation - Transformation)
A Camada Silver é o Coração Analítico do pipeline, onde a lógica de negócio é aplicada e as métricas de Unit Economics são calculadas.

 Justificativas Técnicas
Unificação do Fato: Uso de INNER JOIN entre Pedidos, Entregas e Pagamentos na CTE pedidos_limpos.

Marketplace Contexto: Isso é fundamental, pois garante que a análise do P&L inclua apenas pedidos que completaram as três fases do two-sided marketplace (alocação de pedido, pagamento e logística).

Cálculo da UE: As premissas de negócio (Comissão, Repasse, Taxa Transacional) são definidas em uma CTE constantes e aplicadas via CROSS JOIN.

Reaproveitamento: Esta técnica permite alterar as premissas de negócio (ex: mudar a comissão de 18% para 20%) em um único lugar, sem reescrever o cálculo do P&L.

Limpeza de Dados: Conversão de tipos de dados (CAST(coluna AS DECIMAL(10, 2))) e filtragem de valores inválidos ou não entregues (delivery_status = 'DELIVERED').

#### Etapas & Benefícios

| Métrica Calculada | Fórmula | Benefício Estratégico |
| :--- | :--- | :--- |
| Receita de Comissão | subtotal_bruto * comissao_plataforma | Rentabilidade: Mede o lucro obtido do lado do fornecedor (loja). |
| COGS Logístico Simulado | delivery_fee_cliente * repasse_entregador | Controle de Custo: Mede o custo variável pago ao entregador (lado da logística). |
| Lucro Bruto Unitário | Receita Líquida - COGS Logístico - COGS Transação | Ouro do P&L: A métrica final que indica a Margem de Contribuição de cada pedido. |

#### 4. Camada GOLD: Modelagem Dimensional (Modeling)

A Camada Gold é o resultado final, pronto para o consumo em ferramentas de BI (Power BI, Tableau). É estruturada em um Star Schema para otimizar o desempenho de queries e simplificar a análise.

 Justificativas Técnicas
Star Schema: Separa as métricas (Fato) dos atributos descritivos (Dimensão), acelerando drasticamente os relatórios de BI.

Enriquecimento de Dimensão: O LEFT JOIN na tbl_dim_store_gold (STORES + HUBS) foi crucial.

Contexto: Garante que todas as lojas sejam preservadas (mantendo todos os pedidos da Fato), enquanto anexa a informação geográfica (Cidade/Hub) que estava em outra tabela, criando uma dimensão de loja rica.

Dimensão Tempo (tbl_dim_time_gold): A conversão complexa de STRING para DATE usando TO_TIMESTAMP foi vital para extrair atributos temporais (year, quarter, month) que facilitam análises de tendência e sazonalidade no P&L.

#### Etapas & Benefícios

| Tabela Final | Conteúdo | Benefício para o BI |
| :--- | :--- | :--- |
| tbl_fato_delivery_gold | Métricas de UE e chaves de junção. | Performance: Permite que o Power BI some o lucro_bruto_unitario rapidamente, sem a necessidade de reprocessar cálculos. |
| tbl_dim_store_gold | store_id, store_segment, city. | Segmentação: Permite análises de P&L por segmento de loja (Premium vs. Low) ou por cidade, orientando estratégias de expansão regional. |
| tbl_dim_time_gold | date_key, year, quarter, month. | Tendência: Facilita a análise de séries temporais para identificar padrões de lucratividade (ex: P&L é maior no Q4?). |

### 5. Próximos Passos (Deployment)
O projeto está pronto para a fase de deployment e utilização. A conexão do Power BI às tabelas Gold permite que a equipe de estratégia e finanças monitore o Unit Cost/P&L em tempo real e tome decisões ágeis para otimizar a lucratividade do Marketplace.

In [0]:
%sql
-- NOTEBOOK: 01_ETL_And_Quality.ipynb
-- OBJETIVO: ETL completo + Quality Check em um só notebook

USE CATALOG workspace;
USE pl_delivery_analysis;

-- ===========================================================================
-- LOG INICIAL
-- ===========================================================================

INSERT INTO log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('etl_quality', 'inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando ETL e Quality Check');

SELECT '🚀 INICIANDO ETL COMPLETO + QUALITY CHECK...' as status;

-- ===========================================================================
-- 1. CAMADA BRONZE
-- ===========================================================================

SELECT ' INICIANDO CAMADA BRONZE...' as status;

-- ===========================================================================
-- CAMADA BRONZE - INGESTÃO
-- ===========================================================================

-- Registrar início Bronze
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('bronze', 'ingestao_inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando ingestão Bronze');

-- Criar volume se não existir
CREATE VOLUME IF NOT EXISTS raw_data_volume;

-- -------------------------------------------------------------
-- A. FATO PRINCIPAL (ORDERS)
-- -------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_orders_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_orders_bronze (
  order_id STRING,
  created_at STRING,
  subtotal STRING,
  delivery_fee STRING,
  store_id STRING
);

COPY INTO pl_delivery_analysis.tbl_fact_orders_bronze
FROM (
  SELECT
    order_id,
    order_moment_created AS created_at,
    order_amount AS subtotal,
    order_delivery_fee AS delivery_fee,
    store_id
  FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/orders.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 

-- -----------------------------------------------------------
-- B. FATO LOGÍSTICO (DELIVERIES) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_deliveries_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_deliveries_bronze (
    order_id STRING,
    delivery_status STRING,
    driver_id STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_deliveries_bronze
FROM (
    SELECT
        delivery_order_id AS order_id,
        delivery_status,
        driver_id
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/deliveries.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ','); 

-- -----------------------------------------------------------
-- C. FATO FINANCEIRO (PAYMENTS)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_payments_bronze;
CREATE TABLE pl_delivery_analysis.tbl_fact_payments_bronze (
    order_id STRING,
    payment_method STRING,
    payment_amount STRING
);
COPY INTO pl_delivery_analysis.tbl_fact_payments_bronze
FROM (
    SELECT
        payment_order_id AS order_id,
        payment_method,
        payment_amount
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/payments.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- -----------------------------------------------------------
-- D. DIMENSÃO LOJA (STORES)
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_stores_bronze;
CREATE TABLE pl_delivery_analysis.tbl_dim_stores_bronze (
    store_id STRING,
    store_segment STRING,
    hub_id STRING 
);
COPY INTO pl_delivery_analysis.tbl_dim_stores_bronze
FROM (
    SELECT
        store_id,
        store_segment,
        hub_id
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/stores.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- -----------------------------------------------------------
-- E. DIMENSÃO HUB (HUBS) 
-- -----------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_hubs_bronze;
CREATE TABLE pl_delivery_analysis.tbl_dim_hubs_bronze (
    hub_id STRING,
    city STRING
);
COPY INTO pl_delivery_analysis.tbl_dim_hubs_bronze
FROM (
    SELECT
        hub_id,
        hub_city AS city 
    FROM '/Volumes/workspace/pl_delivery_analysis/raw_data_volume/hubs.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false', 'sep' = ',');

-- Verificação da Camada Bronze
SELECT 
  'Bronze - Orders' as tabela, COUNT(*) as total 
FROM pl_delivery_analysis.tbl_fact_orders_bronze
UNION ALL
SELECT 
  'Bronze - Deliveries', COUNT(*) 
FROM pl_delivery_analysis.tbl_fact_deliveries_bronze
UNION ALL
SELECT 
  'Bronze - Payments', COUNT(*) 
FROM pl_delivery_analysis.tbl_fact_payments_bronze
UNION ALL
SELECT 
  'Bronze - Stores', COUNT(*) 
FROM pl_delivery_analysis.tbl_dim_stores_bronze
UNION ALL
SELECT 
  'Bronze - Hubs', COUNT(*) 
FROM pl_delivery_analysis.tbl_dim_hubs_bronze;

-- Registrar fim Bronze
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
SELECT 
    'bronze', 
    'ingestao_fim', 
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    CONCAT('Bronze carregado: ', 
           (SELECT COUNT(*) FROM tbl_fact_orders_bronze), ' orders, ',
           (SELECT COUNT(*) FROM tbl_fact_deliveries_bronze), ' deliveries, ',
           (SELECT COUNT(*) FROM tbl_fact_payments_bronze), ' payments')
FROM (SELECT 1) t;

-- ===========================================================================
-- 2. CAMADA SILVER  
-- ===========================================================================

SELECT '🔄 INICIANDO CAMADA SILVER...' as status;

-- ===========================================================================
-- CAMADA SILVER - TRANSFORMAÇÃO (COM DEDUPLICAÇÃO)
-- ===========================================================================

-- Registrar início Silver
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('silver', 'transformacao_inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando transformação Silver');

-- Recriar a Silver Layer com deduplicação
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fact_pedidos_silver;

CREATE TABLE pl_delivery_analysis.tbl_fact_pedidos_silver AS
WITH constantes AS (
  SELECT 0.18 AS comissao_plataforma, 0.70 AS repasse_entregador, 0.02 AS taxa_transacao
),

-- DEDUPLICAR deliveries: Para cada order_id, pegar APENAS UM delivery 'DELIVERED'
deliveries_dedup AS (
  SELECT 
    order_id,
    driver_id,
    delivery_status
  FROM (
    SELECT 
      *,
      ROW_NUMBER() OVER (
        PARTITION BY order_id 
        ORDER BY 
          CASE WHEN delivery_status = 'DELIVERED' THEN 1 ELSE 2 END,
          driver_id  -- critério de desempate
      ) as rn
    FROM pl_delivery_analysis.tbl_fact_deliveries_bronze
    WHERE delivery_status = 'DELIVERED'  -- FILTRAR APENAS ENTREGUES
  ) 
  WHERE rn = 1  -- Pegar apenas o primeiro registro por order_id
),

-- DEDUPLICAR payments: Para cada order_id, pegar APENAS UM payment
payments_dedup AS (
  SELECT 
    order_id,
    payment_method,
    CAST(payment_amount AS DECIMAL(10,2)) as payment_amount
  FROM (
    SELECT 
      *,
      ROW_NUMBER() OVER (
        PARTITION BY order_id 
        ORDER BY CAST(payment_amount AS DECIMAL(10,2)) DESC
      ) as rn
    FROM pl_delivery_analysis.tbl_fact_payments_bronze
  ) 
  WHERE rn = 1
),

-- JOIN FINAL: Garantir 1:1 entre orders, deliveries e payments
pedidos_limpos AS (
  SELECT
    o.order_id,
    o.store_id,
    d.driver_id,
    p.payment_method,
    CAST(o.subtotal AS DECIMAL(10, 2)) AS subtotal_bruto,
    CAST(o.delivery_fee AS DECIMAL(10, 2)) AS delivery_fee_cliente,
    o.created_at AS created_at_ts_str
  FROM pl_delivery_analysis.tbl_fact_orders_bronze o
  INNER JOIN deliveries_dedup d ON o.order_id = d.order_id  -- APENAS orders com delivery
  INNER JOIN payments_dedup p ON o.order_id = p.order_id   -- APENAS orders com payment
  WHERE CAST(o.subtotal AS DECIMAL(10, 2)) > 0  -- Garantir subtotal válido
)

SELECT
  p.order_id,
  p.store_id,
  p.driver_id,
  p.payment_method,
  p.subtotal_bruto,
  p.delivery_fee_cliente,
  p.created_at_ts_str,
  -- Métricas de Receita
  (p.subtotal_bruto + p.delivery_fee_cliente) AS gmv_total,
  (p.subtotal_bruto * c.comissao_plataforma) AS receita_comissao,
  (receita_comissao + (p.delivery_fee_cliente * (1 - c.repasse_entregador))) AS receita_liquida_plataforma,
  -- Métricas de Custo
  (p.delivery_fee_cliente * c.repasse_entregador) AS cogs_logistico_simulado,
  ((p.subtotal_bruto + p.delivery_fee_cliente) * c.taxa_transacao) AS cogs_transacao_simulado,
  -- Unit Economics
  (receita_liquida_plataforma - cogs_logistico_simulado - cogs_transacao_simulado) AS lucro_bruto_unitario
FROM pedidos_limpos p
CROSS JOIN constantes c;

-- Verificação da Silver
SELECT 
  'Silver - Pedidos' as tabela, 
  COUNT(*) as total,
  COUNT(DISTINCT order_id) as orders_unicos,
  ROUND(AVG(lucro_bruto_unitario), 2) as margem_media
FROM pl_delivery_analysis.tbl_fact_pedidos_silver;

-- Registrar fim Silver
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
SELECT 
    'silver', 
    'transformacao_fim', 
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    CONCAT('Silver processado: ', 
           (SELECT COUNT(*) FROM tbl_fact_pedidos_silver), ' registros, ',
           (SELECT ROUND(AVG(lucro_bruto_unitario), 2) FROM tbl_fact_pedidos_silver), ' margem média')
FROM (SELECT 1) t;

-- ===========================================================================
-- 3. CAMADA GOLD
-- ===========================================================================

SELECT '🏆 INICIANDO CAMADA GOLD...' as status;

-- ===========================================================================
-- CAMADA GOLD - MODELAGEM DIMENSIONAL
-- ===========================================================================

-- Registrar início Gold
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('gold', 'modelagem_inicio', 'iniciado', CURRENT_TIMESTAMP(), 'Iniciando modelagem Gold');

-- Função de conversão de data
CREATE OR REPLACE FUNCTION convert_datetime_to_date(datetime_str STRING)
RETURNS DATE
RETURN TO_DATE(
    TO_TIMESTAMP(datetime_str, 'M/d/yyyy h:mm:ss a')
);

-- ------------------------------------------------------------
-- 1. DIMENSÃO LOJA (STORES + HUBS)
-- ------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_store_gold;
CREATE TABLE pl_delivery_analysis.tbl_dim_store_gold AS
SELECT 
  s.store_id,
  s.store_segment,
  s.hub_id,
  h.city
FROM pl_delivery_analysis.tbl_dim_stores_bronze s
LEFT JOIN pl_delivery_analysis.tbl_dim_hubs_bronze h ON s.hub_id = h.hub_id;

-- ------------------------------------------------------------
-- 2. DIMENSÃO TEMPO
-- ------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_dim_time_gold;
CREATE TABLE pl_delivery_analysis.tbl_dim_time_gold AS
SELECT DISTINCT
    d.date_key,
    YEAR(d.date_key) AS year,
    MONTH(d.date_key) AS month,
    DAYOFMONTH(d.date_key) AS day,
    WEEKDAY(d.date_key) AS day_of_week_num,
    CASE 
        WHEN MONTH(d.date_key) IN (1, 2, 3) THEN 'Q1'
        WHEN MONTH(d.date_key) IN (4, 5, 6) THEN 'Q2'
        WHEN MONTH(d.date_key) IN (7, 8, 9) THEN 'Q3'
        ELSE 'Q4'
    END AS quarter
FROM (
    SELECT
        convert_datetime_to_date(created_at_ts_str) AS date_key
    FROM pl_delivery_analysis.tbl_fact_pedidos_silver
) d
WHERE d.date_key IS NOT NULL;

-- ------------------------------------------------------------
-- 3. TABELA FATO PRINCIPAL
-- ------------------------------------------------------------
DROP TABLE IF EXISTS pl_delivery_analysis.tbl_fato_delivery_gold;
CREATE TABLE pl_delivery_analysis.tbl_fato_delivery_gold AS
SELECT
    p.order_id,
    p.store_id,
    p.driver_id,
    p.payment_method,
    convert_datetime_to_date(p.created_at_ts_str) AS order_date_key,
    p.subtotal_bruto,
    p.delivery_fee_cliente,
    p.gmv_total,
    p.receita_liquida_plataforma,
    p.cogs_logistico_simulado,
    p.cogs_transacao_simulado,
    p.lucro_bruto_unitario
FROM
    pl_delivery_analysis.tbl_fact_pedidos_silver p;

-- Otimizar a tabela fato
OPTIMIZE pl_delivery_analysis.tbl_fato_delivery_gold
ZORDER BY (order_date_key, store_id);

-- Adicionar comentários
COMMENT ON TABLE pl_delivery_analysis.tbl_fato_delivery_gold IS '
Tabela fato principal com métricas de Unit Economics.
ZORDERed por order_date_key e store_id para performance.
Uso: Análise de P&L e profitability por pedido.
';

-- Verificação da Gold
SELECT 
  'Gold - Fato' as tabela, COUNT(*) as total
FROM pl_delivery_analysis.tbl_fato_delivery_gold
UNION ALL
SELECT 
  'Gold - Dim Store', COUNT(*)
FROM pl_delivery_analysis.tbl_dim_store_gold
UNION ALL
SELECT 
  'Gold - Dim Time', COUNT(*)
FROM pl_delivery_analysis.tbl_dim_time_gold;

-- Registrar fim Gold
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
SELECT 
    'gold', 
    'modelagem_fim', 
    'sucesso', 
    CURRENT_TIMESTAMP(), 
    CONCAT('Gold criado: ',
           (SELECT COUNT(*) FROM tbl_fato_delivery_gold), ' fatos, ',
           (SELECT COUNT(*) FROM tbl_dim_store_gold), ' lojas, ',
           (SELECT COUNT(*) FROM tbl_dim_time_gold), ' datas')
FROM (SELECT 1) t;

-- Finalizar pipeline
INSERT INTO pl_delivery_analysis.log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('pipeline_completo', 'fim', 'sucesso', CURRENT_TIMESTAMP(), 'Pipeline executado com sucesso');

SELECT '✅ Pipeline ETL completo executado com sucesso!' as status;

-- ===========================================================================
-- 4. QUALITY CHECK
-- ===========================================================================

SELECT '📊 INICIANDO QUALITY CHECK...' as status;

USE pl_delivery_analysis;

-- ===========================================================================
-- 1. DASHBOARD DE QUALIDADE DOS DADOS
-- ===========================================================================

WITH quality_metrics AS (
  -- Métricas Bronze
  SELECT 'bronze_orders' as tabela, COUNT(*) as total, COUNT(DISTINCT order_id) as distinct_ids
  FROM tbl_fact_orders_bronze
  UNION ALL
  SELECT 'bronze_deliveries' as tabela, COUNT(*) as total, COUNT(DISTINCT order_id) as distinct_ids
  FROM tbl_fact_deliveries_bronze
  UNION ALL
  SELECT 'bronze_payments' as tabela, COUNT(*) as total, COUNT(DISTINCT order_id) as distinct_ids
  FROM tbl_fact_payments_bronze
  UNION ALL
  -- Métricas Silver
  SELECT 'silver_pedidos' as tabela, COUNT(*) as total, COUNT(DISTINCT order_id) as distinct_ids
  FROM tbl_fact_pedidos_silver
  UNION ALL
  -- Métricas Gold
  SELECT 'gold_fato' as tabela, COUNT(*) as total, COUNT(DISTINCT order_id) as distinct_ids
  FROM tbl_fato_delivery_gold
)

SELECT 
  tabela,
  total,
  distinct_ids,
  CASE 
    WHEN total = distinct_ids THEN '✅ OK'
    WHEN total > distinct_ids THEN '⚠️ DUPLICATAS'
    ELSE '❌ ERRO'
  END as status_chaves,
  ROUND((distinct_ids * 100.0 / total), 2) as percentual_unicos
FROM quality_metrics;

-- ===========================================================================
-- 2. MÉTRICAS DE NEGÓCIO
-- ===========================================================================

SELECT 
  'Métricas Financeiras' as categoria,
  COUNT(*) as total_pedidos,
  ROUND(AVG(lucro_bruto_unitario), 2) as margem_media,
  ROUND(SUM(lucro_bruto_unitario), 2) as lucro_total,
  ROUND(SUM(gmv_total), 2) as gmv_total,
  SUM(CASE WHEN lucro_bruto_unitario < 0 THEN 1 ELSE 0 END) as pedidos_prejuizo,
  ROUND((SUM(CASE WHEN lucro_bruto_unitario < 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) as percentual_prejuizo
FROM tbl_fact_pedidos_silver;

-- ===========================================================================
-- 3. LOGS DE EXECUÇÃO
-- ===========================================================================

SELECT 
  processo,
  etapa,
  status,
  timestamp,
  detalhes
FROM log_processamento 
ORDER BY timestamp DESC
LIMIT 10;

-- ===========================================================================
-- FINALIZAÇÃO
-- ===========================================================================

INSERT INTO log_processamento 
(processo, etapa, status, timestamp, detalhes)
VALUES ('etl_quality', 'fim', 'sucesso', CURRENT_TIMESTAMP(), 'ETL e Quality Check concluídos');

SELECT '🎉 PIPELINE COMPLETO COM SUCESSO!' as status;
SELECT '📊 TABELAS GOLD PRONTAS PARA POWER BI' as info;